In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import logging
import os

In [4]:
data_path = os.path.join('..', 'data', 'raw','politifact_fake.csv')


In [5]:
# Lecture du fichier CSV et conversion en DataFrame
data = pd.read_csv(data_path)


In [6]:
# Affichage du DataFrame
data

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...
...,...,...,...,...
427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,954602090462146560\t954602093171609600\t954650...
428,politifact14071,https://web.archive.org/web/20170322070001/htt...,Lindsey Graham Threatens To Convert To Democra...,740351669834244096\t740391312277573632\t740474...
429,politifact13577,http://archive.is/DQqCx,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO A...,810939651900907520\t814738996706164737\t814917...
430,politifact13560,politicono.com/palin-calls-boycott-mall-americ...,Sarah Palin Calls To Boycott Mall Of America B...,805216591424212993\t805222830023540736\t805223...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [8]:
data_path1 = os.path.join('..', 'data', 'raw','politifact_real.csv')

In [9]:
# Lecture du fichier CSV et conversion en DataFrame
data1 = pd.read_csv(data_path1)

# Affichage du DataFrame
data1

,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...
...,...,...,...,...
619,politifact14731,https://www.flake.senate.gov/public/index.cfm/...,Flake: “Religious tests should have no place i...,NaN
620,politifact329,https://web.archive.org/web/20080131000131/htt...,Change We Can Believe In,634287923135909888\t946743411100536832\t946816...
621,politifact1576,http://www.youtube.com/watch?v=4O8CxZ1OD58,deputy director of national health statistics ...,NaN
622,politifact4720,http://www.youtube.com/watch?v=EhyMplwY6HY,Romneys ProLife Conversion Myth or Reality Jun...,188871706637647874


In [10]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [11]:
# On garde uniquement les trois premières colonnes
data = data.iloc[:, 0:3]
# Affichage du résultat
data

,id,news_url,title
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay
...,...,...,...
427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?
428,politifact14071,https://web.archive.org/web/20170322070001/htt...,Lindsey Graham Threatens To Convert To Democra...
429,politifact13577,http://archive.is/DQqCx,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO A...
430,politifact13560,politicono.com/palin-calls-boycott-mall-americ...,Sarah Palin Calls To Boycott Mall Of America B...


In [12]:
data1 = data1.iloc[:, 0:3]
# Affichage du résultat
data1

,id,news_url,title
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele..."
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008"
...,...,...,...
619,politifact14731,https://www.flake.senate.gov/public/index.cfm/...,Flake: “Religious tests should have no place i...
620,politifact329,https://web.archive.org/web/20080131000131/htt...,Change We Can Believe In
621,politifact1576,http://www.youtube.com/watch?v=4O8CxZ1OD58,deputy director of national health statistics ...
622,politifact4720,http://www.youtube.com/watch?v=EhyMplwY6HY,Romneys ProLife Conversion Myth or Reality Jun...


In [13]:
# Ajout d'une quatrième colonne avec des annotations de Fake_news
annotations = ["Fake_news"] * len(data)
data["Annotations"] = annotations

# Affichage du résultat
data

,id,news_url,title,Annotations
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,Fake_news
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,Fake_news
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,Fake_news
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,Fake_news
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,Fake_news
...,...,...,...,...
427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,Fake_news
428,politifact14071,https://web.archive.org/web/20170322070001/htt...,Lindsey Graham Threatens To Convert To Democra...,Fake_news
429,politifact13577,http://archive.is/DQqCx,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO A...,Fake_news
430,politifact13560,politicono.com/palin-calls-boycott-mall-americ...,Sarah Palin Calls To Boycott Mall Of America B...,Fake_news


In [14]:
# Ajout d'une quatrième colonne avec des annotations de Fake_news
annotations = ["Good_news"] * len(data1)
data1["Annotations"] = annotations

# Affichage du résultat
data1

,id,news_url,title,Annotations
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,Good_news
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,Good_news
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",Good_news
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,Good_news
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",Good_news
...,...,...,...,...
619,politifact14731,https://www.flake.senate.gov/public/index.cfm/...,Flake: “Religious tests should have no place i...,Good_news
620,politifact329,https://web.archive.org/web/20080131000131/htt...,Change We Can Believe In,Good_news
621,politifact1576,http://www.youtube.com/watch?v=4O8CxZ1OD58,deputy director of national health statistics ...,Good_news
622,politifact4720,http://www.youtube.com/watch?v=EhyMplwY6HY,Romneys ProLife Conversion Myth or Reality Jun...,Good_news


In [15]:

# Configuration du logger
logging.basicConfig(filename='example.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Nous vérifions la taille de chaque DataFrame
size_df1 = len(data)
size_df2 = len(data1)

# Ajout de logs pour vérifier les tailles des DataFrames
logging.info(f'Taille de data : {size_df1}')
logging.info(f'Taille de data1 : {size_df2}')

# Si df1 est plus grand ou égal à df2
if size_df1 >= size_df2:
    # On concaténe df à df1 en ajoutant des valeurs manquantes pour les colonnes de df1
    data2 = pd.concat([data, data1.reindex(data.columns, axis=1)], ignore_index=True)
    logging.info('df1 est plus grand ou égal à df2')
else:
    # On concaténe data à data1 en ajoutant des valeurs manquantes pour les colonnes de df1
    data2 = pd.concat([data.reindex(data1.columns, axis=1), data1], ignore_index=True)
    logging.info('df1 est plus petit que df2')


# Mélange des lignes du DataFrame en conservant l'ordre des annotations
np.random.seed(42)  # Pour la reproductibilité du mélange (shuffled)
data_shuffled = data2.sample(frac=1, random_state=42).reset_index(drop=True)

# Affichage du DataFrame fusionné et mélangé 
data_shuffled
logging.info('DataFrame fusionné et mélangé : \n' + str(data_shuffled))

In [16]:
# Suppression des lignes contenant des valeurs manquantes
data_without_missing = data_shuffled.dropna()

# Affichage des DataFrames sans les valeurs manquantes
data_without_missing

,id,news_url,title,Annotations
0,politifact14885,www.breakingnews247.net/59bc13819a723/world-s-...,World's most popular candy to be removed from ...,Fake_news
1,politifact1106,http://frwebgate.access.gpo.gov/cgi-bin/getdoc...,Browse Congressional Bills,Good_news
2,politifact1678,http://abcnews.go.com/ThisWeek/video/supreme-c...,Supreme Court Vacancy Video,Good_news
3,politifact370,http://tonto.eia.doe.gov/dnav/pet/pet_move_wkl...,U.S. Imports & Exports,Good_news
4,politifact15263,http://sciencevibe.com/2018/04/11/dying-78-yea...,Dying 78 Year Old CIA Agent Admits To Killing ...,Fake_news
...,...,...,...,...
1051,politifact14090,http://breaking13news.com/malia-obama-arrested...,Malia Obama Arrested With A Gang Of Thugs In C...,Fake_news
1052,politifact6641,http://www.bls.gov/ces/,"Employment, Hours, and Earnings from the Curre...",Good_news
1053,politifact14904,https://web.archive.org/web/20180109225217/htt...,HOLLYWOOD CELEBS: WE WILL GO ON ‘TOTAL STRIKE’...,Fake_news
1054,politifact4555,http://www.cq.com/doc/newsmakertranscripts-395...,testimony submitted to the Congressional Overs...,Good_news


In [17]:
# Renommage de la colonne "Titre de la fake news" en "Title_news"

data_without_missing.rename(columns={'title': 'Title_news'}, inplace=True)

# Affichage du DataFrame avec la colonne renommée
data_without_missing

C:\Users\user\AppData\Local\Temp\ipykernel_14220\3530571739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_without_missing.rename(columns={'title': 'Title_news'}, inplace=True)


,id,news_url,Title_news,Annotations
0,politifact14885,www.breakingnews247.net/59bc13819a723/world-s-...,World's most popular candy to be removed from ...,Fake_news
1,politifact1106,http://frwebgate.access.gpo.gov/cgi-bin/getdoc...,Browse Congressional Bills,Good_news
2,politifact1678,http://abcnews.go.com/ThisWeek/video/supreme-c...,Supreme Court Vacancy Video,Good_news
3,politifact370,http://tonto.eia.doe.gov/dnav/pet/pet_move_wkl...,U.S. Imports & Exports,Good_news
4,politifact15263,http://sciencevibe.com/2018/04/11/dying-78-yea...,Dying 78 Year Old CIA Agent Admits To Killing ...,Fake_news
...,...,...,...,...
1051,politifact14090,http://breaking13news.com/malia-obama-arrested...,Malia Obama Arrested With A Gang Of Thugs In C...,Fake_news
1052,politifact6641,http://www.bls.gov/ces/,"Employment, Hours, and Earnings from the Curre...",Good_news
1053,politifact14904,https://web.archive.org/web/20180109225217/htt...,HOLLYWOOD CELEBS: WE WILL GO ON ‘TOTAL STRIKE’...,Fake_news
1054,politifact4555,http://www.cq.com/doc/newsmakertranscripts-395...,testimony submitted to the Congressional Overs...,Good_news


In [18]:
# Enregistrement du DataFrame en fichier CSV
output_path = os.path.join('..', 'data', 'processed', 'mabs.csv')


In [19]:
data_without_missing.to_csv(output_path, index=False)
